# MATRICE DI CORRELAZIONE

Come prima operazione, importiamo le librerie necessarie

In [1]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import pandas as pd
import numpy as np

Successivamente si procede con l'importazione del dataset contenuto nella cartella "data". Per raggiungere tale scopo si utilizza il metodo **read_excel()** del modulo **pandas**

In [2]:
table = pd.read_excel('https://github.com/collab-uniba/PanelJamDataExtractor/blob/master/data/TabellaCompleta.xlsx?raw=true', index = False)

Una volta importato il dataset si devono eliminare alcune feature contenute in esso, questo perchè:

-le feature **id_prog**,**id_panel**,**autore_panel** contengono delle stringhe che indicano rispettivamente, l'id dei progetti estratti, l'id dei panel contenuti in ogni progetto e il nome del creatore di ogni panel. Queste feature non sono rilevanti per il calcolo della matrice di correlazione(mostrata successivamente) e nemmeno per la costruzione del modello di regressione.

-le feature **Likes**, **Views**, **Comments** e **time** fanno riferimento ai progetti e non ai singoli panel, per questo motivo non vengono considerati.

Si procede con la costruzione della matrice di correlazione con lo scopo di osservare eventuali problemi di **multicollinearità** tra le feature. Se il grado di correlazione tra due feature è maggiore del limite consentito di **0.7**, se ne deve eliminare una delle due. La matrice verrà costruita utilizzando il metodo **corr()** del modulo **pandas**

In [3]:
#time = table['time'].tolist()
#table['time'] = list(map(lambda sub:int(''.join([ele for ele in sub if ele.isnumeric()])), time))
table = table.drop(columns = ['id_prog','id_panel','Likes','Views','Comments','autore_panel','time'])

corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,project_depth,remixed,stars_autore,has_avatar_autore,has_bio_autore,followers_autore,tot_loves_autore,tot_views_autore,ranking_autore,panel_stars
project_depth,1.00,-0.42,0.05,0.06,0.03,0.02,0.04,0.05,0.06,0.07
remixed,-0.42,1.00,-0.03,-0.03,-0.03,-0.02,-0.03,-0.03,-0.03,-0.17
stars_autore,0.05,-0.03,1.00,0.09,-0.04,0.55,0.74,0.85,0.45,0.17
has_avatar_autore,0.06,-0.03,0.09,1.00,0.24,0.18,0.15,0.11,0.13,0.08
has_bio_autore,0.03,-0.03,-0.04,0.24,1.00,0.22,0.12,0.01,-0.08,0.06
followers_autore,0.02,-0.02,0.55,0.18,0.22,1.00,0.71,0.63,0.35,0.18
tot_loves_autore,0.04,-0.03,0.74,0.15,0.12,0.71,1.00,0.76,0.32,0.19
tot_views_autore,0.05,-0.03,0.85,0.11,0.01,0.63,0.76,1.00,0.64,0.12
ranking_autore,0.06,-0.03,0.45,0.13,-0.08,0.35,0.32,0.64,1.00,0.16
panel_stars,0.07,-0.17,0.17,0.08,0.06,0.18,0.19,0.12,0.16,1.00


Come si può osservare dalla matrice, tra la feature **tot_views_autore** e **stars_autore** c'è un elevato grado di correlazione (**0.85**), questo è dovuto dalla presenza di entrambe le feature nel calcolo del ranking di un autore. Dato che star_count rappresenta la misura di apprezzamento migliore, si decide di eliminare tot_views_autore. A supporto di questa decisione è la presenza di un alto grado di correlazione tra tot_views_autore e **tot_loves_autore** (**0.76**).

Nonostante il grado di correlazione tra tot_loves_autore e stars_autore sia di **0.74**, esso è comunque nel limite tollerabile. 
Inoltre, questo valore è dovuto dalla presenza di tot_loves_autore nel calcolo del ranking. Di seguito viene mostrata la matrice di correlazione corretta.

In [4]:
table = table.drop(columns = ['tot_views_autore'])
corrMatrix = table.corr()
corrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

,project_depth,remixed,stars_autore,has_avatar_autore,has_bio_autore,followers_autore,tot_loves_autore,ranking_autore,panel_stars
project_depth,1.00,-0.42,0.05,0.06,0.03,0.02,0.04,0.06,0.07
remixed,-0.42,1.00,-0.03,-0.03,-0.03,-0.02,-0.03,-0.03,-0.17
stars_autore,0.05,-0.03,1.00,0.09,-0.04,0.55,0.74,0.45,0.17
has_avatar_autore,0.06,-0.03,0.09,1.00,0.24,0.18,0.15,0.13,0.08
has_bio_autore,0.03,-0.03,-0.04,0.24,1.00,0.22,0.12,-0.08,0.06
followers_autore,0.02,-0.02,0.55,0.18,0.22,1.00,0.71,0.35,0.18
tot_loves_autore,0.04,-0.03,0.74,0.15,0.12,0.71,1.00,0.32,0.19
ranking_autore,0.06,-0.03,0.45,0.13,-0.08,0.35,0.32,1.00,0.16
panel_stars,0.07,-0.17,0.17,0.08,0.06,0.18,0.19,0.16,1.00


# MODELLO DI REGRESSIONE

Dopo aver costruito la matrice di correlazione ed eliminato eventuali problemi di collinearità, si procede con la costruzione del modello di regressione. Il primo passaggio consiste nel dividere il dataset in due, dove il primo conterrà solo le feature non numeriche ed è stato etichettato con il nome **logit**, il secondo conterrà solo le feature numeriche ed è stato etichettato con il nome **numerical**. In seguito si spiegherà il motivo di questa divisione.

In [6]:
featureTable = table
logit = featureTable.drop(columns = ['project_depth','stars_autore','followers_autore','tot_loves_autore',
                'ranking_autore','panel_stars'])
numerical = featureTable.drop(columns = ['remixed','has_avatar_autore','has_bio_autore'])

La feature **project_depth** all'interno della tabella, sta ad indicare la profondità di ogni progetto, ovvero il numero di vignette contenute in esso. Se, per una vignetta, è presente quella successiva allora il valore di **extended** relativa ad essa sarà True, altrimenti False. In questa maniera, se un progetto dovesse avere solo una vignetta(ovvero quella del creatore del progetto stesso), la project_depth relativa avrà valore 1. Al fine di ottenere una corretta costruzione del modello di regressione, il valore della project_depth di un progetto non esteso, dovrà necessariamente essere pari a 0, per questo motivo si è deciso di sottrare 1 alle profondità di tutti i progetti.

In [7]:
numerical['project_depth'] = numerical['project_depth'] - 1

bisogna effettuare una log-normalizzazione delle feature numeriche in quanto alcune di esse contengono valori troppo grandi. Dopo di chè si procede alla ricostruzione della tabella, andando ad unire la tabella numerica(ottenuta dalla log-normalizzazione) alla tabella non numerica.

In [8]:
normTable = np.log1p(numerical)
normTable['remixed'] = logit['remixed']
normTable['has_avatar_autore'] = logit['has_avatar_autore']
normTable['has_bio_autore'] = logit['has_bio_autore']

Per costruire il modello di regressione si è utilizzata la funzione **glm** del package **statsmodel**, alla quale sono stati passati in input 3 parametri.

-**formula**: prima del simbolo tilde (~) si deve specificare la variabile dipendente, mentre dopo bisogna specificare le variabili indipendenti, i quali costituiranno i predittori del modello.

-**data**: la tabella da cui estrarre i valori delle variabili indipendenti.

-**family**: specifica il modello di regressione che si vuole andare a costruire, in questo caso binomial ed è specificato da **sm.families.Binomial()**.

In [9]:
formula = """remixed ~ stars_autore + project_depth + followers_autore + tot_loves_autore +
             has_bio_autore + has_avatar_autore + ranking_autore + panel_stars"""

model = smf.glm(formula = formula, data = normTable, family=sm.families.Binomial(statsmodels.genmod.families.links.logit)).fit()
print(model.summary())

                          Generalized Linear Model Regression Results                          
Dep. Variable:     ['remixed[False]', 'remixed[True]']   No. Observations:                13748
Model:                                             GLM   Df Residuals:                    13739
Model Family:                                 Binomial   Df Model:                            8
Link Function:                                   logit   Scale:                          1.0000
Method:                                           IRLS   Log-Likelihood:                -5249.2
Date:                                 Fri, 29 May 2020   Deviance:                       10498.
Time:                                         18:24:11   Pearson chi2:                 1.31e+04
No. Iterations:                                      6                                         
Covariance Type:                             nonrobust                                         
                                coef    

d:\dev-cpp\anaconda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  after removing the cwd from sys.path.
